In [11]:
import os
import pandas as pd
import numpy as np
import datetime as dt
import random
from scipy.stats import poisson 

In [12]:
folder = 'C:\Projects\Bike Hire\Weekly 2022'
df = []
for file in os.listdir(folder):
    if file.endswith('.csv'):
        print('Loading file {0}...'.format(file))
        df.append(pd.read_csv(os.path.join(folder,file)))
journey = pd.concat(df)
journey.head()

Loading file week1.csv...
Loading file week10.csv...
Loading file week11.csv...
Loading file week12.csv...
Loading file week13.csv...
Loading file week14.csv...
Loading file week15.csv...
Loading file week16.csv...
Loading file week17.csv...
Loading file week18.csv...
Loading file week19.csv...
Loading file week2.csv...
Loading file week20.csv...
Loading file week21.csv...
Loading file week22.csv...
Loading file week23.csv...
Loading file week24.csv...
Loading file week25.csv...
Loading file week26.csv...
Loading file week27.csv...
Loading file week3.csv...
Loading file week4.csv...
Loading file week5.csv...
Loading file week6.csv...
Loading file week7.csv...
Loading file week8.csv...
Loading file week9.csv...


,Rental Id,Duration,Bike Id,End Date,EndStation Id,EndStation Name,Start Date,StartStation Id,StartStation Name,EndStation Latitude,EndStation Longitude,StartStation Latitude,StartStation Longitude
0,115967515,1260,15338,01-01-2022 23:13,310,"Black Prince Road, Vauxhall",01-01-2022 22:52,529,"Manresa Road, Chelsea",51.4909,-0.116912,51.4869,-0.170983
1,116017034,720,19861,04-01-2022 19:08,11,"Brunswick Square, Bloomsbury",04-01-2022 18:56,804,"Good's Way, King's Cross",51.5240,-0.122502,51.5347,-0.125079
2,116016563,480,19861,04-01-2022 18:46,804,"Good's Way, King's Cross",04-01-2022 18:38,57,"Guilford Street , Bloomsbury",51.5347,-0.125079,51.5233,-0.120203
3,116014412,1260,17235,04-01-2022 17:45,14,"Belgrove Street , King's Cross",04-01-2022 17:24,297,"Geraldine Street, Elephant & Castle",51.5299,-0.123617,51.4961,-0.106001
4,116013350,480,13790,04-01-2022 16:50,252,"Jubilee Gardens, South Bank",04-01-2022 16:42,310,"Black Prince Road, Vauxhall",51.5046,-0.116542,51.4909,-0.116912


In [13]:
stations = pd.read_csv('C:\Projects\Bike Hire\stations 2022.csv')
stations

,Docking Station Name,Docking Station ID,Docking Station Capacity,Latitude,Longitude
0,"River Street , Clerkenwell",1,19,51.529200,-0.109971
1,"Phillimore Gardens, Kensington",2,37,51.499600,-0.197574
2,"Christopher Street, Liverpool Street",3,32,51.521300,-0.084606
3,"St. Chad's Street, King's Cross",4,23,51.530100,-0.120974
4,"Sedding Street, Sloane Square",5,27,51.493100,-0.156876
...,...,...,...,...,...
795,"George Row, Bermondsey",840,22,51.498585,-0.068982
796,"Tower Wharf, Bermondsey",841,24,51.500845,-0.074704
797,"Temple Gardens, Temple",842,24,51.510981,-0.108323
798,"Canada Water Station, Rotherhithe",844,35,51.498439,-0.049150


In [14]:
journey['Start Date Time'] = pd.to_datetime(journey['Start Date'], format= '%d-%m-%Y %H:%M')
journey['End Date Time'] = pd.to_datetime(journey['Start Date'], format= '%d-%m-%Y %H:%M')
journey['Start Hour'] = journey['Start Date Time'].dt.hour
journey['End Hour'] = journey['End Date Time'].dt.hour

In [16]:
#mean average arrivals per station per time period
arrivals = journey.groupby(['EndStation Id', 'End Date', 'End Hour'], as_index=False)['Rental Id'].count()\
                .groupby(['EndStation Id', 'End Hour'])['Rental Id'].mean().round(2).unstack(level=0)
arrivals

EndStation Id,1,2,3,4,5,6,7,8,9,10,...,834,835,836,838,839,840,841,842,844,845
End Hour,,,,,,,,,,,,,,,,,,,,,
0,1.19,1.13,1.04,1.23,1.19,1.13,1.06,1.16,1.26,1.27,...,1.11,1.22,1.13,1.07,1.29,1.21,1.09,1.33,1.11,1.07
1,1.36,1.14,1.06,1.10,1.34,1.08,1.22,1.11,1.18,1.33,...,1.09,1.52,1.22,1.00,1.39,1.08,1.11,1.43,1.18,1.24
2,1.14,1.24,1.23,1.11,1.23,1.00,1.24,1.15,1.30,1.56,...,1.17,1.36,1.12,1.00,1.56,1.18,1.16,1.30,1.19,1.03
3,1.40,1.29,1.20,1.23,1.18,1.00,1.17,1.13,1.12,1.43,...,1.23,1.43,1.07,1.33,1.23,1.14,1.15,1.25,1.17,1.17
4,1.00,1.20,1.09,1.33,1.06,1.00,1.38,1.24,1.09,1.00,...,1.00,1.00,1.26,1.00,1.10,1.07,1.14,1.10,1.14,1.12
5,1.00,1.00,1.00,1.06,1.03,1.02,1.00,1.10,1.15,1.14,...,1.05,1.10,1.21,1.00,1.04,1.06,1.00,1.06,1.05,1.02
6,1.00,1.06,1.03,1.05,1.05,1.02,1.00,1.06,1.00,1.05,...,1.03,1.03,1.00,1.02,1.04,1.00,1.04,1.01,1.01,1.01
7,1.00,1.06,1.03,1.05,1.11,1.06,1.05,1.08,1.03,1.07,...,1.08,1.09,1.03,1.11,1.03,1.02,1.14,1.05,1.02,1.02
8,1.02,1.05,1.11,1.03,1.12,1.10,1.03,1.03,1.09,1.07,...,1.07,1.10,1.02,1.11,1.11,1.03,1.05,1.12,1.03,1.04


In [17]:
#mean average departures per station per time period
departures = journey.groupby(['StartStation Id', 'Start Date', 'Start Hour'], as_index=False)['Rental Id'].count()\
                .groupby(['StartStation Id', 'Start Hour'])['Rental Id'].mean().round(2).unstack(level=0)
departures

StartStation Id,1,2,3,4,5,6,7,8,9,10,...,834,835,836,838,839,840,841,842,844,845
Start Hour,,,,,,,,,,,,,,,,,,,,,
0,1.19,1.14,1.16,1.16,1.05,1.15,1.09,1.15,1.12,1.10,...,1.05,1.03,1.15,1.09,1.22,1.15,1.16,1.25,1.19,1.18
1,1.29,1.14,1.11,1.16,1.23,1.18,1.47,1.12,1.48,1.14,...,1.04,1.14,1.12,1.10,1.11,1.06,1.17,1.17,1.16,1.07
2,1.18,1.03,1.17,1.17,1.08,1.10,1.12,1.04,1.44,1.19,...,1.00,1.19,1.27,1.50,1.14,1.19,1.15,1.28,1.42,1.10
3,1.14,1.06,1.31,1.09,1.14,1.06,1.20,1.00,1.38,1.62,...,1.05,1.15,1.28,1.00,1.41,1.05,1.17,1.26,1.31,1.38
4,1.14,1.40,1.17,1.17,1.11,1.08,1.00,1.00,1.15,1.17,...,1.10,1.07,1.09,1.00,1.29,1.00,1.06,1.07,1.38,1.05
5,1.00,1.17,1.00,1.25,1.05,1.00,1.00,1.03,1.00,1.25,...,1.00,1.22,1.12,1.00,1.06,1.04,1.00,1.14,1.00,1.00
6,1.01,1.03,1.00,1.04,1.03,1.03,1.01,1.02,1.01,1.00,...,1.00,1.06,1.05,1.00,1.01,1.01,1.01,1.02,1.03,1.03
7,1.04,1.06,1.05,1.08,1.07,1.01,1.03,1.05,1.01,1.01,...,1.05,1.07,1.03,1.04,1.03,1.05,1.04,1.03,1.07,1.03
8,1.04,1.08,1.02,1.08,1.06,1.04,1.06,1.10,1.02,1.01,...,1.10,1.01,1.04,1.14,1.05,1.09,1.05,1.09,1.10,1.07


In [18]:
capacity = stations.groupby(['Docking Station ID'])['Docking Station Capacity'].max()
arrival_rate = arrivals.loc[0:23].mean()
departure_rate = departures.loc[0:23].mean()

In [19]:
def list(arrival_rate, departure_rate):
  arrival_list = ['arrival' for i in range(np.random.poisson(arrival_rate))]
  departure_list = ['departure' for i in range(np.random.poisson(departure_rate))]

  arrival_list.extend(departure_list)
  np.random.shuffle(arrival_list)
  return arrival_list

In [20]:
def experience(arrival_rate, departure_rate, bikes, capacity):
  bad_experience = 0
  for i in list(arrival_rate, departure_rate):
    if i =='arrival':
      bikes += 1
      if bikes > capacity:
        bad_experience += 1 
        bikes -= 1
    if i == 'departure':
      bikes -= 1
      if bikes < 0:
        bad_experience +=1
        bikes += 1
  return bad_experience

In [21]:
df = pd.DataFrame({'arrivals': arrival_rate, 'departures': departure_rate, 'capacity': capacity})
df[np.isnan(df)] = 0
df

,arrivals,departures,capacity
1,1.095833,1.082500,19
2,1.138333,1.123750,37
3,1.115000,1.092917,32
4,1.118750,1.132083,23
5,1.139583,1.105000,27
...,...,...,...
840,1.101667,1.097500,22
841,1.130000,1.107500,24
842,1.265833,1.207083,24
844,1.140417,1.166250,35


In [23]:
experience_data = []
for station in df.T:
  optimal_starting_bikes = 0
  fewest_bad_exp = 1000

  print(f'station: {station}')
  for i in range(int(df['capacity'][station])+1):
    simulations = []
    for simulation in range(100):
      result = experience(df['arrivals'][station], df['departures'][station], i, df['capacity'][station])
      simulations.append(result)

    columns = ['Simulation Results']
    Bad_Experiences_Expected = pd.DataFrame(columns=columns, data=simulations)

    mean_bad_experiences = Bad_Experiences_Expected.mean()[0]
    experience_data.append(mean_bad_experiences)
    if mean_bad_experiences < fewest_bad_exp:
      optimal_starting_bikes = i
      fewest_bad_exp = mean_bad_experiences

    print(f'The number of bad experiences expected with {i} starting bikes is: {mean_bad_experiences}')
  print(f'The optimal number of bikes is {optimal_starting_bikes}')
 
  total_bikes = []
  total_bikes.append(optimal_starting_bikes)
  total = 0
  for i in range(len(total_bikes)):
    total = total + total_bikes[i]
  print(f'Total number of optimal bikes is {total}')

station: 1
The number of bad experiences expected with 0 starting bikes is: 0.66
The number of bad experiences expected with 1 starting bikes is: 0.28
The number of bad experiences expected with 2 starting bikes is: 0.11
The number of bad experiences expected with 3 starting bikes is: 0.04
The number of bad experiences expected with 4 starting bikes is: 0.0
The number of bad experiences expected with 5 starting bikes is: 0.0
The number of bad experiences expected with 6 starting bikes is: 0.0
The number of bad experiences expected with 7 starting bikes is: 0.0
The number of bad experiences expected with 8 starting bikes is: 0.0
The number of bad experiences expected with 9 starting bikes is: 0.0
The number of bad experiences expected with 10 starting bikes is: 0.0
The number of bad experiences expected with 11 starting bikes is: 0.0
The number of bad experiences expected with 12 starting bikes is: 0.0
The number of bad experiences expected with 13 starting bikes is: 0.0
The number of b

C:\Users\diksh\AppData\Local\Temp\ipykernel_24152\761158829.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mean_bad_experiences = Bad_Experiences_Expected.mean()[0]


The number of bad experiences expected with 36 starting bikes is: 0.33
The number of bad experiences expected with 37 starting bikes is: 0.55
The optimal number of bikes is 4
Total number of optimal bikes is 4
station: 3
The number of bad experiences expected with 0 starting bikes is: 0.7
The number of bad experiences expected with 1 starting bikes is: 0.16
The number of bad experiences expected with 2 starting bikes is: 0.05
The number of bad experiences expected with 3 starting bikes is: 0.01
The number of bad experiences expected with 4 starting bikes is: 0.0
The number of bad experiences expected with 5 starting bikes is: 0.0
The number of bad experiences expected with 6 starting bikes is: 0.0
The number of bad experiences expected with 7 starting bikes is: 0.0
The number of bad experiences expected with 8 starting bikes is: 0.0
The number of bad experiences expected with 9 starting bikes is: 0.0
The number of bad experiences expected with 10 starting bikes is: 0.0
The number of ba